In [1]:
import json
from pyahp import parse
import pandas as pd
import numpy as np

# Load DataFrame
df = pd.read_excel('IBM Chatbot User Feedback (Responses).xlsx')

## Data Transformation

In [2]:
# Define a mapping dictionary
value_mapping = {1:1, 2:3, 3:5, 4:7, 5:9}

# Specify the columns I want to transform
cols_to_transform = ['Proactivity VS. Conscientiousness (Extent of Importance)', 'Proactivity VS. Damage control (Extent of Importance)', 'Proactivity VS. Thoroughness (Extent of Importance)', 'Proactivity VS. Emotional intelligence (Extent of Importance)','Proactivity VS. Identity (Extent of Importance)','Proactivity VS. Personality (Extent of Importance)','Conscientiousness VS. Damage control (Extent of Importance)','Conscientiousness VS. Thoroughness (Extent of Importance)','Conscientiousness VS. Emotional intelligence (Extent of Importance)','Conscientiousness VS. Identity (Extent of Importance)','Conscientiousness VS. Personality (Extent of Importance)','Damage control VS. Thoroughness (Extent of Importance)','Damage control VS. Emotional intelligence (Extent of Importance)','Damage control VS. Identity (Extent of Importance)','Damage control VS. Personality (Extent of Importance)','Thoroughness VS. Emotional intelligence (Extent of Importance)','Thoroughness VS. Identity (Extent of Importance)','Thoroughness VS. Personality (Extent of Importance)','Emotional intelligence VS. Identity (Extent of Importance)','Emotional intelligence VS. Personality (Extent of Importance)','Identity VS. Personality (Extent of Importance)']

# Create a copy of the original DataFrame
converted_IBM_response = df.copy()

# Apply the mapping to the specified columns
converted_IBM_response[cols_to_transform] = converted_IBM_response[cols_to_transform].replace(value_mapping)

# Now converted_IBM_response has the values in the specified columns converted according to my mapping
converted_IBM_response

# Fliter irelevant columns
response = converted_IBM_response.drop(columns=['IBM chatbot failed to fulfill', 'Timestamp','Suggestions for IBM Chatbot\'s features or functionalities','Additional Info'])
response.to_csv('response.csv', index=False)

In [3]:
VS_columns = list(filter(lambda x: 'VS' in x and not '(' in x, response.columns))
VS_columns_extent = list(filter(lambda x: 'VS' in x and '(' in x, response.columns))

In [4]:
response.head(5)

,Interacted with IBM Chatbot,Age:,Gender:,Educational Background,Familiarity with emerging technologies,Frequency of using Chatbots,Overall satisfaction (IBM Chatbot),Proactivity VS. Conscientiousness,Proactivity VS. Conscientiousness (Extent of Importance),Proactivity VS. Damage control,...,Thoroughness VS. Identity,Thoroughness VS. Identity (Extent of Importance),Thoroughness VS. Personality,Thoroughness VS. Personality (Extent of Importance),Emotional intelligence VS. Identity,Emotional intelligence VS. Identity (Extent of Importance),Emotional intelligence VS. Personality,Emotional intelligence VS. Personality (Extent of Importance),Identity VS. Personality,Identity VS. Personality (Extent of Importance)
0,Yes,18 - 24 years old,Female,Master's Degree,4,5,3,Proactivity,9,Proactivity,...,Thoroughness,9,Thoroughness,9,Emotional Intelligence,9,Emotional Intelligence,9,Personality,7
1,Yes,18 - 24 years old,Female,Master's Degree,3,4,2,Conscientiousness,3,Proactivity,...,Thoroughness,1,Thoroughness,3,Emotional Intelligence,3,Emotional Intelligence,5,Identity,1
2,Yes,18 - 24 years old,Male,Bachelor's Degree,3,1,5,Conscientiousness,9,Damage control,...,Thoroughness,9,Thoroughness,9,Emotional Intelligence,9,Emotional Intelligence,9,Identity,9
3,Yes,18 - 24 years old,Female,Master's Degree,4,5,4,Conscientiousness,7,Damage control,...,Thoroughness,7,Thoroughness,7,Emotional Intelligence,5,Emotional Intelligence,5,Identity,7
4,Yes,25-34 years old,Male,Ph.D. or higher,5,5,2,Conscientiousness,7,Proactivity,...,Thoroughness,7,Thoroughness,7,Emotional Intelligence,7,Emotional Intelligence,7,Personality,1


In [5]:
response.columns

Index(['Interacted with IBM Chatbot', 'Age: ', 'Gender: ',
       'Educational Background', 'Familiarity with emerging technologies',
       'Frequency of using Chatbots', 'Overall satisfaction (IBM Chatbot)',
       'Proactivity VS. Conscientiousness',
       'Proactivity VS. Conscientiousness (Extent of Importance)',
       'Proactivity VS. Damage control',
       'Proactivity VS. Damage control (Extent of Importance)',
       'Proactivity VS. Thoroughness',
       'Proactivity VS. Thoroughness (Extent of Importance)',
       'Proactivity VS. Emotional intelligence',
       'Proactivity VS. Emotional intelligence (Extent of Importance)',
       'Proactivity VS. Identity',
       'Proactivity VS. Identity (Extent of Importance)',
       'Proactivity VS. Personality',
       'Proactivity VS. Personality (Extent of Importance)',
       'Conscientiousness VS. Damage control',
       'Conscientiousness VS. Damage control (Extent of Importance)',
       'Conscientiousness VS. Thoroughness'

In [8]:
extent_df = response[VS_columns_extent]

In [9]:
def convert_to_csv(response):
    for index, row in response.iterrows():
        for i in range(len(VS_columns)):
            vs_names = VS_columns[i].split(' VS. ')
            if row[VS_columns[i]] != vs_names[0]:
                extent_df.at[index, VS_columns_extent[i]] = -extent_df.at[index, VS_columns_extent[i]]

    
    column_mapping = {}
    for i in range(len(VS_columns_extent)):
        vs_names = VS_columns[i].split(' VS. ')
        new_col = vs_names[0] + '_' + vs_names[1]
        column_mapping[VS_columns_extent[i]] = new_col
    
    extent_df_renamed = extent_df.rename(columns=column_mapping)
    return extent_df_renamed

In [10]:
response['Gender: '].value_counts()

Female               66
Male                 52
Prefer not to say     1
Non-binary            1
Name: Gender: , dtype: int64

In [11]:
extent_df_renamed = convert_to_csv(response)
extent_df_renamed

,Proactivity_Conscientiousness,Proactivity_Damage control,Proactivity_Thoroughness,Proactivity_Emotional intelligence,Proactivity_Identity,Proactivity_Personality,Conscientiousness_Damage control,Conscientiousness_Thoroughness,Conscientiousness_Emotional intelligence,Conscientiousness_Identity,...,Damage control_Thoroughness,Damage control_Emotional intelligence,Damage control_Identity,Damage control_Personality,Thoroughness_Emotional intelligence,Thoroughness_Identity,Thoroughness_Personality,Emotional intelligence_Identity,Emotional intelligence_Personality,Identity_Personality
0,9,9,1,7,9,9,-7,7,7,7,...,7,9,9,9,9,9,9,-9,-9,-7
1,-3,3,-1,3,5,3,3,5,3,5,...,1,3,5,3,1,1,3,-3,-5,1
2,-9,-9,-9,-9,-9,9,9,9,-9,9,...,9,-9,9,9,-9,9,9,-9,-9,9
3,-7,-9,-9,5,7,7,-9,-9,7,5,...,9,9,9,9,7,7,7,-5,-5,7
4,-7,7,7,1,7,7,5,-7,7,9,...,-7,7,9,9,3,7,7,-7,-7,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,-7,3,7,5,7,7,5,7,7,7,...,7,-7,5,7,-5,-5,3,-5,-3,5
116,-5,7,5,-7,5,-7,7,5,9,5,...,7,7,7,9,7,7,5,-7,-7,7
117,9,7,7,-5,5,-5,7,7,9,7,...,7,-7,7,7,7,9,1,-7,-7,7
118,-5,-7,-5,5,7,7,7,7,5,7,...,7,7,3,7,-7,7,7,-7,-5,-7


In [13]:
# 'Interacted with IBM Chatbot', 'Age: ', 'Gender: ',
#        'Educational Background', 'Familiarity with emerging technologies',
#        'Frequency of using Chatbots', 'Overall satisfaction (IBM Chatbot)',
user_profile_columns = ['Interacted with IBM Chatbot', 'Age: ', 'Gender: ',
       'Educational Background', 'Familiarity with emerging technologies',
       'Frequency of using Chatbots', 'Overall satisfaction (IBM Chatbot)']
new_names = ['interact', 'age', 'gender', 'edu', 'fam', 'freq', 'sat']
user_profiles = response[user_profile_columns].rename(columns=dict(zip(user_profile_columns, new_names)))

user_profiles['sat'].value_counts()

4    52
3    47
2    17
5     3
1     1
Name: sat, dtype: int64

In [14]:
extent_df_renamed.to_csv('extents.csv', index=False)
user_profiles.to_csv('user_profiles.csv', index=False)

In [15]:
alternatives = set()
alternatives_pairs = list(map(lambda x: x.split(' VS. '), VS_columns))
for pair in alternatives_pairs:
    alternatives.add(pair[0])
    alternatives.add(pair[1])
alternatives = list(alternatives)
alternatives

['Identity',
 'Proactivity',
 'Thoroughness',
 'Emotional intelligence',
 'Personality',
 'Damage control',
 'Conscientiousness']

In [16]:
response[VS_columns_extent]

,Proactivity VS. Conscientiousness (Extent of Importance),Proactivity VS. Damage control (Extent of Importance),Proactivity VS. Thoroughness (Extent of Importance),Proactivity VS. Emotional intelligence (Extent of Importance),Proactivity VS. Identity (Extent of Importance),Proactivity VS. Personality (Extent of Importance),Conscientiousness VS. Damage control (Extent of Importance),Conscientiousness VS. Thoroughness (Extent of Importance),Conscientiousness VS. Emotional intelligence (Extent of Importance),Conscientiousness VS. Identity (Extent of Importance),...,Damage control VS. Thoroughness (Extent of Importance),Damage control VS. Emotional intelligence (Extent of Importance),Damage control VS. Identity (Extent of Importance),Damage control VS. Personality (Extent of Importance),Thoroughness VS. Emotional intelligence (Extent of Importance),Thoroughness VS. Identity (Extent of Importance),Thoroughness VS. Personality (Extent of Importance),Emotional intelligence VS. Identity (Extent of Importance),Emotional intelligence VS. Personality (Extent of Importance),Identity VS. Personality (Extent of Importance)
0,9,9,1,7,9,9,7,7,7,7,...,7,9,9,9,9,9,9,9,9,7
1,3,3,1,3,5,3,3,5,3,5,...,1,3,5,3,1,1,3,3,5,1
2,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
3,7,9,9,5,7,7,9,9,7,5,...,9,9,9,9,7,7,7,5,5,7
4,7,7,7,1,7,7,5,7,7,9,...,7,7,9,9,3,7,7,7,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,7,3,7,5,7,7,5,7,7,7,...,7,7,5,7,5,5,3,5,3,5
116,5,7,5,7,5,7,7,5,9,5,...,7,7,7,9,7,7,5,7,7,7
117,9,7,7,5,5,5,7,7,9,7,...,7,7,7,7,7,9,1,7,7,7
118,5,7,5,5,7,7,7,7,5,7,...,7,7,3,7,7,7,7,7,5,7


In [17]:
def create_alternative_matrix(row):
    alt_num = len(alternatives)
    alternative_matrix = [[None for i in range(alt_num)] for i in range(alt_num)]
    for i in range(alt_num):
        for j in range(i, alt_num):
            alt_i = alternatives[i]
            alt_j = alternatives[j]
            numerical_choice_column = list(filter(
                lambda x: alt_i in x and alt_j in x, 
                VS_columns_extent
            ))[0]
            binary_choice_column = list(filter(
                lambda x: alt_i in x and alt_j in x, 
                VS_columns
            ))[0]
            binary_choice = row[binary_choice_column]
            numerical_choice = row[numerical_choice_column]
            if binary_choice == alt_i:
                alternative_matrix[i][j] = numerical_choice
            else:
                alternative_matrix[i][j] = 1 / numerical_choice
            if i == j:
                alternative_matrix[i][j] = 1
            
            alternative_matrix[j][i] = 1 / alternative_matrix[i][j]
    return alternative_matrix

def create_criteria_matrix(df):
    return [[1 for i in range(len(df))] for j in range(len(df))]

def create_preference_matrices(df):
    preference_matrices = {}
    preference_matrices['criteria'] = create_criteria_matrix(df)
    for index, row in df.iterrows():
        alternative_matrix = create_alternative_matrix(row)
        preference_matrices['alternatives:{}'.format(index)] = alternative_matrix
    return preference_matrices

def get_priorities(df, method):
    model_dict = {
        'name': 'model',
        'method': method,
        'criteria': [str(x) for x in range(len(df))],
        'subCriteria': {},
        'alternatives': alternatives,
        'preferenceMatrices': create_preference_matrices(df)
    }

    for name, matrix in model_dict['preferenceMatrices'].items():
        if not np.all(np.diag(matrix) == 1):
            print('invalid diagonal')

        det = np.linalg.det(matrix)
        is_singular = abs(det) < 1e-10
        if is_singular:
            print('singular')
            print(name)
            # print(matrix)

    ahp_model = parse(model_dict)
    priorities = ahp_model.get_priorities()
    return priorities


In [18]:
get_priorities(df=response, method='approximate')

singular
criteria
singular
alternatives:16
singular
alternatives:51
singular
alternatives:56
singular
alternatives:62
singular
alternatives:110
singular
alternatives:111


array([0.044, 0.218, 0.077, 0.1  , 0.127, 0.196, 0.238])

In [19]:
get_priorities(df=response, method='geometric')

singular
criteria
singular
alternatives:16
singular
alternatives:51
singular
alternatives:56
singular
alternatives:62
singular
alternatives:110
singular
alternatives:111


array([ 45.141, 314.001,  87.905,  81.82 , 122.028, 246.476, 382.112])

In [39]:
get_priorities(df=response, method='eigenvalue')

singular
criteria
singular
alternatives:16
singular
alternatives:51
singular
alternatives:56
singular
alternatives:62
singular
alternatives:110
singular
alternatives:111


ValueError: matrix type must be 'f', 'd', 'F', or 'D'

## Results
The python package pyAHP fails to compute the correct records using the eigenvalue method.

By carefully checking the source code, I found that this is because the algorithm requires computing eigen values and eigen vectors of alternative matrices, as well as the weight matrix for criteria. However, as shown above, some alternative matrices could be singular, thus is illly defined for eigen vectors.

Especially that for the criteria weight matrix, under the case where each criterion is equally weighted, the criteria matrix must be a matrix of `ones`, which can be proved to always be a singular matrix.

This exploration indicates that the python package pyAHP does not properly handle these singular matrix situations.